In [1]:
from datetime import datetime as dt
from time import sleep
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceOrderException
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor
from secrets import secrets

In [2]:
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
price = {'FTMUSDT': 0, 'error': False}
portfolio = {'Initial': 600, 'USDT': 600, 'FTM': 0, 'BuyPrice': 0, 'SellPrice': 0, 'TransTime': dt.now(),
             'TransFee': 0.001, 'TradeLimit': 0.0015}

In [3]:
def ftm_pairs_trade(msg):
    if msg['e'] != 'error':
        price['FTMUSDT'] = float(msg['c'])
    else:
        price['error'] = True

def buy_ftm(price):
    portfolio['FTM'] = portfolio['USDT'] / price * (1 - portfolio['TransFee'])
    portfolio['USDT'] = 0
    portfolio['BuyPrice'] = price
    portfolio['TransTime'] = dt.now()
    print('{}\tBUY @ {}, FTM: {}, USDT: {}'.format(dt.now(), price, portfolio['FTM'], portfolio['USDT']))

def sell_ftm(price):
    portfolio['USDT'] = portfolio['FTM'] * price * (1 - portfolio['TransFee'])
    portfolio['FTM'] = 0
    portfolio['SellPrice'] = price
    portfolio['TransTime'] = dt.now()
    print('{}\tSELL @ {}, FTM: {}, USDT: {}'.format(dt.now(), price, portfolio['FTM'], portfolio['USDT']))
    
def percent_diff(curr, prev):
    return (curr - prev) / prev

In [4]:
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
bsm = BinanceSocketManager(client)
conn_key = bsm.start_symbol_ticker_socket('FTMUSDT', ftm_pairs_trade)
bsm.start()

In [5]:
sleep(1)
buy_ftm(price['FTMUSDT'])

while True:
    if price['error']:
        bsm.stop_socket(conn_key)
        bsm.start()
        price['error'] = False
    else:
        curr_price = price['FTMUSDT']
        if portfolio['FTM'] > 0:
            if percent_diff(curr_price, portfolio['BuyPrice']) > portfolio['TradeLimit']:
                sell_ftm(curr_price)
        else:
            if percent_diff(curr_price, portfolio['SellPrice']) < -1 * portfolio['TradeLimit']:
                buy_ftm(curr_price)
            elif (dt.now() - portfolio['TransTime']).seconds > 60 * 60:
                buy_ftm(curr_price)

2021-02-28 00:16:59.024783	BUY @ 0.47366, FTM: 1265.4646793058312, USDT: 0
2021-02-28 00:17:00.531994	SELL @ 0.47448, FTM: 0, USDT: 599.8372433559937
2021-02-28 00:17:05.374257	BUY @ 0.4735, FTM: 1265.5489041449582, USDT: 0


KeyboardInterrupt: 

In [6]:
bsm.stop_socket(conn_key)
reactor.stop()

In [19]:
t = client.get_ticker(symbol='FTMUSDT')
t

{'symbol': 'FTMUSDT',
 'priceChange': '-0.08671000',
 'priceChangePercent': '-15.448',
 'weightedAvgPrice': '0.50943035',
 'prevClosePrice': '0.56200000',
 'lastPrice': '0.47460000',
 'lastQty': '6234.90000000',
 'bidPrice': '0.47436000',
 'bidQty': '2400.00000000',
 'askPrice': '0.47499000',
 'askQty': '3936.20000000',
 'openPrice': '0.56131000',
 'highPrice': '0.63000000',
 'lowPrice': '0.44500000',
 'volume': '553801855.20000000',
 'quoteVolume': '282123471.53681900',
 'openTime': 1614356622896,
 'closeTime': 1614443022896,
 'firstId': 12001451,
 'lastId': 12414145,
 'count': 412695}

In [17]:
portfolio

{'Initial': 600,
 'USDT': 0,
 'FTM': 1265.5489041449582,
 'BuyPrice': 0.4735,
 'SellPrice': 0.47448,
 'TransTime': datetime.datetime(2021, 2, 28, 0, 17, 5, 374256),
 'TransFee': 0.001,
 'TradeLimit': 0.0015}

In [21]:
PORTFOLIO_TEMPLATE = """
Portfolio
USDT:\t\t{}
FTM:\t\t{}
    \t\t@ ${}
Networth:\t{}
Net Growth:\t{}
Percent Growth:\t{}%
"""

if portfolio['USDT'] > 0:
    networth = portfolio['USDT']
else:
    networth = portfolio['FTM'] * float(t['lastPrice'])

net_growth = networth - portfolio['Initial']
percent_growth = net_growth / portfolio['Initial'] * 100

print(PORTFOLIO_TEMPLATE.format(portfolio['USDT'], portfolio['FTM'], t['lastPrice'], networth, net_growth, percent_growth))


Portfolio
USDT:		0
FTM:		1265.5489041449582
    		@ $0.47460000
Networth:	600.6295099071972
Net Growth:	0.629509907197189
Percent Growth:	0.10491831786619817%



In [34]:
INITIAL = 600
PORTFOLIO = {'USDT': INITIAL, 'FTM': 0, 'BuyPrice': 0, 'SellPrice': 0, 'TransTime': dt.now(), 'TransFee': 0.001,
             'TradeLimit': 0.0015}

def scout_market(coin):
    symbol = coin + 'USDT'
    client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
    curr_price = float(client.get_ticker(symbol=symbol)['lastPrice'])
    if PORTFOLIO['FTM'] > 0:
        if percent_diff(curr_price, PORTFOLIO['BuyPrice']) > PORTFOLIO['TradeLimit']:
            sell_ftm(curr_price)
    else:
        if percent_diff(PORTFOLIO['SellPrice'], curr_price) > PORTFOLIO['TradeLimit']:
            buy_ftm(curr_price)
        elif (dt.now() - PORTFOLIO['TransTime']).seconds > 60 * 30:
            buy_ftm(curr_price)

def buy_ftm(price):
    PORTFOLIO['FTM'] = PORTFOLIO['USDT'] / price * (1 - PORTFOLIO['TransFee'])
    PORTFOLIO['USDT'] = 0
    PORTFOLIO['BuyPrice'] = price
    PORTFOLIO['TransTime'] = dt.now()
    print('{}\tBUY @ {}, FTM: {}, USDT: {}'.format(dt.now(), price, PORTFOLIO['FTM'], PORTFOLIO['USDT']))

def sell_ftm(price):
    PORTFOLIO['USDT'] = PORTFOLIO['FTM'] * price * (1 - PORTFOLIO['TransFee'])
    PORTFOLIO['FTM'] = 0
    PORTFOLIO['SellPrice'] = price
    PORTFOLIO['TransTime'] = dt.now()
    print('{}\tSELL @ {}, FTM: {}, USDT: {}'.format(dt.now(), price, PORTFOLIO['FTM'], PORTFOLIO['USDT']))
    
def percent_diff(curr, prev):
    return (curr - prev) / prev

In [35]:
client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
curr_price = float(client.get_ticker(symbol='FTMUSDT')['lastPrice'])
buy_ftm(curr_price)
PORTFOLIO

2021-02-28 00:45:31.851286	BUY @ 0.473, FTM: 1267.23044397463, USDT: 0


{'USDT': 0,
 'FTM': 1267.23044397463,
 'BuyPrice': 0.473,
 'SellPrice': 0,
 'TransTime': datetime.datetime(2021, 2, 28, 0, 45, 31, 851276),
 'TransFee': 0.001,
 'TradeLimit': 0.0015}

In [36]:
scout_market('FTM')